In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import callback_context
from dash.dependencies import Input, Output
import base64

prompter: object = None

app = JupyterDash()

@app.callback(
    [Output('okay-button', 'disabled'),
     Output('back-button', 'disabled'),
     Output('reset-button', 'disabled')],
    Input('button-status-update', 'n_intervals')
)
def _update_button_status(n_intervals):
    return prompter._update_button_status()

@app.callback(
    Output('prompter-output', 'children'),
    [Input('okay-button', 'n_clicks'),
     Input('back-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')]
)
def _update_prompter_output(n_clicks_okay, n_clicks_back, n_clicks_reset):
    if not callback_context.triggered:
        button = None
    else:
        button = callback_context.triggered[0]['prop_id'].split('.')[0]
    return prompter._update_prompter_output(button)

class Prompter(object):
    def __init__(self, mode='inline', port=8050, width='100%', height='650px', prompts=[], callbacks=[], allow_back=True):
        self.mode = mode
        self.port = port
        self.width = width
        self.height = height
        self.prompts = prompts
        self.callbacks = callbacks
        self.allow_back = allow_back

        self.step = 0
        self.steps = len(self.prompts)
        self.running_callback = False
        
        self.back_button_image = '/usr/local/syna/lib/python/plotter/assets/back-svgrepo-com.png'
        self.back_button_image = base64.b64encode(open(self.back_button_image, 'rb').read())

        self.reset_button_image = '/usr/local/syna/lib/python/plotter/assets/reload-svgrepo-com.png'
        self.reset_button_image = base64.b64encode(open(self.reset_button_image, 'rb').read())

        assert self.steps is not 0, 'Empty prompt list'

        global prompter
        assert prompter is None, 'Attempted to instantiate more than one prompter. Please restart kernel.'
        prompter = self
        global app
        self.app = app
        self.app.layout = html.Div(
            id = 'prompter-container',
            children = [
                html.Div(
                    id = 'prompter-output',
                    children = self.prompts[0],
                    style = {
                        'margin': 'auto',
                        'marginBottom': '10px'
                    }
                ),
                html.Div(
                    id = 'buttons',
                    children = [
                        html.Button(
                            id = 'back-button',
                            title = 'Back',
                            children = [
                                html.Img(
                                    src = 'data:image/png;base64,{}'.format(self.back_button_image.decode()),
                                    width = '15px',
                                    height = '15px',
                                    style = {
                                        'paddingTop': '3px'
                                    }
                                )
                            ],
                            disabled = True,
                            n_clicks = 0,
                            style = {
                                'width': '50px',
                                'height': '30px',
                                'cursor': 'pointer'
                            }
                        ),
                        html.Button(
                            'Okay',
                            id = 'okay-button',
                            title = 'Okay',
                            n_clicks = 0,
                            style = {
                                'width': '150px',
                                'height': '30px',
                                'cursor': 'pointer'
                            }
                        ),
                        html.Button(
                            id = 'reset-button',
                            title = 'Reset',
                            children = [
                                html.Img(
                                    src = 'data:image/png;base64,{}'.format(self.reset_button_image.decode()),
                                    width = '15px',
                                    height = '15px',
                                    style = {
                                        'paddingTop': '3px'
                                    }
                                )
                            ],
                            n_clicks = 0,
                            style = {
                                'width': '50px',
                                'height': '30px',
                                'cursor': 'pointer'
                            }
                        )
                    ],
                    style = {
                        'display': 'flex',
                        'flexDirection': 'row',
                        'justifyContent': 'center',
                        'columnGap': '20px'
                    }
                ),
                dcc.Interval(
                    id = 'button-status-update',
                    interval = 100,
                    n_intervals = 0
                )
            ],
            style = {
                'marginTop': '30px',
                'fontFamily': 'Arial',
                'fontSize': '16px',
                'display': 'flex',
                'flexDirection': 'column',
                'rowGap': '10px',
                'whiteSpace': 'nowrap'
            }
        )

    def _update_button_status(self):
        if self.running_callback:
            return True, True, True
        elif self.step == 0:
            return False, True, False
        elif self.step >= self.steps:
            if self.allow_back:
                return True, False, False
            else:
                return True, True, False
        else:
            if self.allow_back:
                return False, False, False
            else:
                return False, True, False

    def _on_click_okay_button(self):
        self.step += 1

        if self.step <= len(self.callbacks):
            callback = self.callbacks[self.step - 1]
            if callback is not None:
                self.running_callback = True
                callback()
                self.running_callback = False

        if self.step >= self.steps:
            return 'End of prompt messages'

        return self.prompts[self.step]

    def _on_click_back_button(self):
        if self.step > 0:
            self.step -= 1
        return self.prompts[self.step]

    def _on_click_reset_button(self):
        self.step = 0
        return self.prompts[self.step]

    def _update_prompter_output(self, button):
        if button is None:
            return self.prompts[self.step]
        elif button == 'okay-button':
            return self._on_click_okay_button()
        elif button == 'back-button':
            return self._on_click_back_button()
        elif button == 'reset-button':
            return self._on_click_reset_button()
        else:
            return self.prompts[self.step]

    def run(self):
        self.app.run_server(mode = self.mode, host = 'localhost', port = self.port, width = self.width, height = self.height)
